In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q alibi

     |████████████████████████████████| 397 kB 14.6 MB/s 
     |████████████████████████████████| 3.5 MB 53.7 MB/s 
     |████████████████████████████████| 28.0 MB 1.3 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 6.8 MB 49.2 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import pandas as pd
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from alibi.explainers import IntegratedGradients
print('Eager execution enabled: ', tf.executing_eagerly()) # True

Eager execution enabled:  True


In [ ]:
train_x = pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_train_all.csv')
test_x =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_test_all.csv')

train_y = pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/y_train_all.csv')
test_y =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/y_test_all.csv')

y_test = test_y.rate
y_train = train_y.rate


In [ ]:
train_sentences = train_x.clean_review.values.astype(str)
test_sentences = test_x.clean_review.values.astype(str)

In [ ]:
max_features = 90166
max_length = 50

In [ ]:
model_lstm = load_model('/content/drive/MyDrive/MasterThesis/LSTM/lstm.h5')

In [ ]:
def decode_sentence(x, reverse_index):
    return " ".join([reverse_index.get(i, 'UNK') for i in x])

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/word_index', 'rb') as f:
    word_index = pickle.load(f)
reverse_index = {value: key for (key, value) in word_index.items()}    

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/train_padded', 'rb') as f:
    train_padded = pickle.load(f)
    
with open('/content/drive/MyDrive/MasterThesis/Dataset/test_padded', 'rb') as f:
    test_padded = pickle.load(f)    

**IG**

In [ ]:
layer = model_lstm.layers[1]
layer

In [ ]:
n_steps = 50
method = "gausslegendre"
internal_batch_size = 100
nb_samples = 20
ig  = IntegratedGradients(model_lstm,
                          layer=layer,
                          n_steps=n_steps,
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
#function to remove UNK

def func_remove_unk(b):
  l = list(filter(lambda x: x[0] != 'UNK', b))
  return l

In [ ]:
def func_sum (sample):
  x = sample
  d = {}
  [d.__setitem__(first, d.get(first, 0) + second) for first, second in x]
  r = list(d.items())
  return r

In [ ]:
def func (val):
  x_test = test_padded 
  j = val
  x_test_sample = x_test[j:j+2] #2476
  predictions = np.round( model_lstm(x_test_sample).numpy())
  explanation = ig.explain(x_test_sample,
                         baselines=None,
                         target=predictions)

  attrs = explanation.attributions[0]
  attrs = attrs.sum(axis=2)
  #Get attributions values from the explanation object


  i = 0
  x_i = x_test_sample[i]
  attrs_i = attrs[i]
  words = decode_sentence(x_i, reverse_index).split()


  a=words
  b=attrs_i

  temp_list = []
  for i in range(0, len(a)):
    temp_list.append((a[i],b[i]))
    l = list(filter(lambda x: x[0] != 'UNK', temp_list))
    l = func_sum (l)
  sorted_list = sorted(l, key=lambda x: x[1])
  return predictions[0], sorted_list

In [ ]:
pos_i = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/positive_sample_50', 'rb'))
neg_i = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/negative_sample_50', 'rb')) 

In [ ]:
resN=[]
for el in neg_i:
  r = func (el)
  resN.append(r)

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/ig_cnn_final_list_neg', 'wb') as f:
    pickle.dump(resN, f)    

In [ ]:
resP=[]
for el in pos_i:
  r = func (el)
  resP.append(r)

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/ig_cnn_final_list_pos', 'wb') as f:
    pickle.dump(resP, f)    

In [ ]:
#Get attributions values from the explanation object
i = 0
x_i = x_test_sample[i]
attrs_i = attrs[i]
words = decode_sentence(x_i, reverse_index).split()


a=words
b=attrs_i

temp_list = []
for i in range(0, len(a)):
  temp_list.append((a[i],b[i]))
sorted_list = sorted(temp_list, key=lambda x: x[1])
sorted_list

In [ ]:
#predictions =  np.concatenate( predictions, axis=0 )
#predictions[0]


In [ ]:
# Metadata from the explanation object
#explanation.meta

In [ ]:
# Data fields from the explanation object
#explanation.data.keys()

In [ ]:
#Get attributions values from the explanation object
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

In [ ]:
i = 0
x_i = x_test_sample[i]
attrs_i = attrs[i]
pred = predictions[i]
pred_dict = {1: 'Positive review', 0: 'Negative review'}

In [ ]:
from IPython.display import HTML
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"

In [ ]:
def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    import matplotlib as mpl
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)

    # now compute hex values of colors
    colors = list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))
    return colors

In [ ]:
words = decode_sentence(x_i, reverse_index).split()
colors = colorize(attrs_i)

In [ ]:
print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))
HTML("".join(list(map(hlstr, words, colors))))

In [ ]:
attrs_i 

In [ ]:
#ig values calculated for all test sample
x_test = test_padded
x_test_sample = x_test
predictions = np.round( model_lstm(x_test_sample).numpy())
explanation = ig.explain(x_test_sample,
                         baselines=None,
                         target=predictions)

In [ ]:
attrs = explanation.attributions[0]
attrs = attrs.sum(axis=2)


#we need this information to use later for the survey
import pickle
with open('/content/drive/MyDrive/MasterThesis/Dataset/ig_values_lstm', 'wb') as f:
  pickle.dump(attrs, f)    

**Sample calculation**

Calculating and saving feature importance score for the reviews used as a demonstration example to measure the proposed criteria.

In [ ]:
pos_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/positive_sample_50', 'rb'))
neg_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/negative_sample_50', 'rb')) 

In [ ]:
r_ig= pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/ig_values_lstm', 'rb'))  
r_ig[0]  

In [ ]:
#function to remove UNK

def func_remove_unk(b):
  l = list(filter(lambda x: x[0] != 'UNK', b))
  return l

In [ ]:
#match each word with its value
def match_words(vec): #vec = list of indexes
  ig_values = []
  for i in vec:
    temp = []
    x_i = test_padded[i]
    attrs_i = r_ig[i]
    words = decode_sentence(x_i, reverse_index).split()
    for indx in range(len(words)):
      temp.append((words[indx], attrs_i[indx]))
    ig_values.append(temp)
  return ig_values

In [ ]:
i = pos_sample[0]
test_padded[i]
r_ig[i]
decode_sentence(x_i, reverse_index).split()
i

In [ ]:
p = match_words(pos_sample)
n = match_words(neg_sample)

In [ ]:
def func_sum (sample):
  my_sum = []
  for i in range(len(sample)):
    x = sample[i]
    d = {}
    [d.__setitem__(first, d.get(first, 0) + second) for first, second in x]
    r = list(d.items())
    my_sum.append(r)
  return my_sum

In [ ]:
resp = func_sum(p)
resn = func_sum(p)

In [ ]:
r1 = []
for el in resp:
  a = func_remove_unk(el)
  a_sorted = sorted(a, key=lambda x: x[1])
  r1.append(a_sorted)

r2 = []
for el in resn:
  a = func_remove_unk(el)
  a_sorted = sorted(a, key=lambda x: x[1])
  r2.append(a_sorted)

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/ig_lstm_final_list_pos', 'wb') as f:
    pickle.dump(r1, f)

with open('/content/drive/MyDrive/MasterThesis/Dataset/ig_lstm_final_list_neg', 'wb') as f:
    pickle.dump(r2, f)     